In [2]:
import numpy as np
import pandas as pd

In [22]:
import re
import math
import json
import datetime

In [23]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [47]:
# Alternatively, if you are not running the Notebook in Google Colab

# from google.colab import drive 
# drive.mount('/content/drive/')
# listings = pd.read_csv('drive/MyDrive/com-480-cam/data/vaud/listings-detailed.csv')

geneva = pd.read_csv('../data/geneva/listings-detailed.csv')
geneva['region'] = 'Geneva'

vaud = pd.read_csv('../data/vaud/listings-detailed.csv')
vaud['region'] = 'Vaud'

zurich = pd.read_csv('../data/zurich/listings-detailed.csv')
zurich['region'] = 'Zurich'

listings = pd.concat([geneva, vaud, zurich])

print(f'Concatenated pd.DataFrame of shape: {listings.shape}')

Concatenated pd.DataFrame of shape: (8222, 75)


In [27]:
# Convert dates and datetimes to pandas.DateTime
def format_dates(df: pd.DataFrame, feature: str, format: str = '%Y-%m-%d'):
    df[feature] = pd.to_datetime(df[feature], format=format)
    
# Format price by removing commas and dollar sign
def format_price(price: str):
    return float(price[1:].replace(',', ''))

# Classify property type
def find_type(property_type: str):
    
    if 'entire' in property_type.lower():
        return 'place'
    
    if 'room' in property_type.lower():
        return 'room'
    
    return 'other'

# Time since date
def days_since(d: pd.Timestamp) -> float:

    today = datetime.datetime.now()
    d = d.to_pydatetime()

    return (today - d).days

In [48]:
dated_features = ['last_scraped', 'host_since', 'calendar_last_scraped', 
                  'first_review', 'last_review']

timestamped_features = ['scrape_id']

for feature in dated_features:
    format_dates(listings, feature)
    
for feature in timestamped_features:
    format_dates(listings, feature, format='%Y%m%d%H%M%S')
    
listings['price'] = listings.price.apply(format_price)
listings['amenities_count'] = listings.amenities.apply(lambda a: len(a))
listings['type'] = listings.property_type.apply(find_type)
listings['days_host'] = listings.host_since.apply(days_since)
listings['days_host'] = listings.days_host.apply(lambda d: 0 if math.isnan(d) else int(d))
listings['host_acceptance_rate'] = listings.host_acceptance_rate.apply(lambda r: float(str(r).strip('%')))

In [8]:
listings.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [49]:
filtered = listings[['neighbourhood_cleansed', 'bedrooms', 'beds', 'accommodates', 'amenities_count', 'maximum_nights', 'minimum_nights', 'host_acceptance_rate', 'bathrooms', 'review_scores_value', 'price']][listings.price < 500].copy()

In [50]:
filtered.head()

,neighbourhood_cleansed,bedrooms,beds,accommodates,amenities_count,maximum_nights,minimum_nights,host_acceptance_rate,bathrooms,review_scores_value,price
0,Commune de Genève,1.0,1.0,1,286,365,30,0.0,NaN,9.0,72.0
1,Commune de Genève,1.0,1.0,1,795,90,1,100.0,NaN,9.0,90.0
2,Commune de Genève,1.0,1.0,1,791,1125,6,100.0,NaN,9.0,75.0
3,Commune de Genève,1.0,2.0,4,360,1125,14,92.0,NaN,9.0,135.0
4,Versoix,1.0,3.0,5,621,360,5,96.0,NaN,9.0,80.0


In [51]:
table = pd.pivot_table(
    filtered,
    values=['neighbourhood_cleansed', 'bedrooms', 'beds', 'accommodates', 'amenities_count', 'maximum_nights', 'minimum_nights', 'host_acceptance_rate', 'bathrooms', 'review_scores_value', 'price'],
    index='neighbourhood_cleansed',
    aggfunc=np.mean
)

table['neighbourhood'] = table.index

table.reset_index(drop=True, inplace=True)

In [52]:
table

,accommodates,amenities_count,bedrooms,beds,host_acceptance_rate,maximum_nights,minimum_nights,price,review_scores_value,neighbourhood
0,4.666667,377.000000,2.000000,2.666667,98.500000,754.666667,3.333333,123.000000,9.500000,Aclens
1,1.827586,370.517241,1.107143,1.518519,71.894737,667.620690,6.724138,80.517241,9.523810,Affoltern
2,2.514286,416.914286,1.214286,1.328571,98.059701,1029.771429,1.185714,79.742857,9.571429,Aigle
3,2.225806,307.193548,1.166667,1.233333,64.750000,456.806452,5.548387,77.838710,9.304348,Albisrieden
4,4.000000,387.000000,1.666667,3.000000,100.000000,755.000000,1.333333,182.333333,9.000000,Allaman
...,...,...,...,...,...,...,...,...,...,...
290,2.745763,324.813559,1.230769,1.474576,77.260870,549.762712,4.694915,110.966102,9.533333,Wollishofen
291,7.000000,238.000000,4.000000,6.000000,NaN,180.000000,2.000000,99.000000,9.000000,Yens
292,2.771429,384.742857,1.379310,1.685714,77.500000,520.314286,3.571429,77.257143,9.535714,Yverdon-les-Bains
293,4.200000,289.400000,2.200000,2.400000,90.000000,472.200000,3.400000,123.800000,9.000000,Yvonand


In [53]:
table.to_csv('../airbnb-visualized/data/listings-filtered.csv', index=False)